# Traffic fatalities vs. homicides, per year

In [1]:
from utz import *
from utz.colors import colors_lengthen
from utz.plots import save
import plotly.express as px
from nj_crashes.paths import HOMICIDES_PQT
from nj_crashes.colors import get_colors, gridcolor

In [2]:
dot = read_parquet('../njdot/cm.pqt')
dot

,Date,County,Total Injured,Property Damage,Total Killed
index,,,,,
0,2001-01-01,ATLANTIC,340,494,2
1,2001-01-01,BERGEN,1053,2765,1
2,2001-01-01,BURLINGTON,347,818,6
3,2001-01-01,CAMDEN,525,957,0
4,2001-01-01,CAPE MAY,65,102,1
...,...,...,...,...,...
247,2020-12-01,SALEM,44,100,2
248,2020-12-01,SOMERSET,133,492,0
249,2020-12-01,SUSSEX,53,209,0


In [3]:
dot_yrs = dot.groupby(dot.Date.dt.year)['Total Killed'].sum()
dot_yrs

Date
2001    726
2002    609
2003    752
2004    738
2005    751
2006    745
2007    710
2008    613
2009    577
2010    567
2011    609
2012    575
2013    538
2014    551
2015    588
2016    625
2017    684
2018    622
2019    587
2020    611
Name: Total Killed, dtype: int64

In [4]:
homicides = read_parquet(HOMICIDES_PQT).homicides
cmp = (
    sxs(
        dot_yrs.rename('fatalities'),
        homicides
    )
    .sort_index()
    .dropna(subset='fatalities')
    .astype(int)
)
years = cmp.index
years.name = 'year'
cmp

,fatalities,homicides
year,,
2001,726,336
2002,609,339
2003,752,406
2004,738,392
2005,751,417
2006,745,428
2007,710,380
2008,613,376
2009,577,319


In [5]:
traffic_homicide_ratio = round((cmp.fatalities / cmp.homicides).rename('ratio'), 2)
traffic_homicide_ratio

year
2001    2.16
2002    1.80
2003    1.85
2004    1.88
2005    1.80
2006    1.74
2007    1.87
2008    1.63
2009    1.81
2010    1.53
2011    1.60
2012    1.48
2013    1.33
2014    1.57
2015    1.62
2016    1.68
2017    2.11
2018    2.18
2019    2.24
2020    1.86
Name: ratio, dtype: float64

In [6]:
from IPython.display import Markdown, display

def swatches(colors, sep=' ', width=6):
    display(Markdown(sep.join(
        f'<span style="font-family: monospace">{color} <span style="color: {color}">{chr(9608)*width}</span></span>'
        for color in colors
    )))    

In [7]:
colors = get_colors(len(years))
black, red, year_colors = colors.black, colors.red, colors.year_colors

swatches(year_colors)

<span style="font-family: monospace">#fcffa4 <span style="color: #fcffa4">██████</span></span> <span style="font-family: monospace">#f9e973 <span style="color: #f9e973">██████</span></span> <span style="font-family: monospace">#f7d342 <span style="color: #f7d342">██████</span></span> <span style="font-family: monospace">#f8ba25 <span style="color: #f8ba25">██████</span></span> <span style="font-family: monospace">#faa00b <span style="color: #faa00b">██████</span></span> <span style="font-family: monospace">#f58811 <span style="color: #f58811">██████</span></span> <span style="font-family: monospace">#ef7020 <span style="color: #ef7020">██████</span></span> <span style="font-family: monospace">#e35d2f <span style="color: #e35d2f">██████</span></span> <span style="font-family: monospace">#d54b3f <span style="color: #d54b3f">██████</span></span> <span style="font-family: monospace">#c33d4c <span style="color: #c33d4c">██████</span></span> <span style="font-family: monospace">#b03259 <span style="color: #b03259">██████</span></span> <span style="font-family: monospace">#9b2862 <span style="color: #9b2862">██████</span></span> <span style="font-family: monospace">#862168 <span style="color: #862168">██████</span></span> <span style="font-family: monospace">#70196c <span style="color: #70196c">██████</span></span> <span style="font-family: monospace">#5a116b <span style="color: #5a116b">██████</span></span> <span style="font-family: monospace">#450c66 <span style="color: #450c66">██████</span></span> <span style="font-family: monospace">#2e0c52 <span style="color: #2e0c52">██████</span></span> <span style="font-family: monospace">#190b3d <span style="color: #190b3d">██████</span></span> <span style="font-family: monospace">#0c0520 <span style="color: #0c0520">██████</span></span> <span style="font-family: monospace">#000004 <span style="color: #000004">██████</span></span>

In [8]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
years = cmp.index.tolist()
y2 = 'Ratio: Crashes / Homicides'
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Bar(x=cmp.index, y=cmp.fatalities, name='Traffic deaths', marker_color=red))
fig.add_trace(go.Bar(x=cmp.index, y=cmp.homicides, name='Homicides', marker_color=black))
fig.add_trace(go.Scatter(x=traffic_homicide_ratio.index, y=traffic_homicide_ratio, name=y2, line=dict(width=6, color='#f99708'), mode='lines'), secondary_y=True)
save(
    fig, title='NJ Traffic Deaths vs. Homicides', name='crash_homicide_cmp',
    bg='white',
    hoverx=True,
    xaxis=dict(
        tickmode='array',
        tickvals=years,
        ticktext=years,
        tickangle=-45,
        fixedrange=True,
    ),    
    ytitle='Deaths',
    yaxis=dict(
        fixedrange=True,
    ),
    yaxis2=dict(
        title=y2,
        gridcolor='#ccc',
        fixedrange=True,
    ),
);

Wrote image to ../www/public/plots/crash_homicide_cmp.png


![](../www/public/plots/crash_homicide_cmp.png)